In [5]:
import time 
import requests, bs4
import re
import sys
import os
import datetime
from urllib.parse import urlparse

import pandas as pd
import numpy as np
from omegaconf import OmegaConf
import pandas_gbq

sys.path.append("../")
from src.scraping import ScrapingBase, ScrapingSponavi

%matplotlib inline
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 出力先
conf_dir = "../config"
conf_cli = OmegaConf.from_cli()
conf_exec = OmegaConf.load(os.path.join(conf_dir, "config_exec.yaml"))
conf_path = OmegaConf.load(os.path.join(conf_dir, "config_path.yaml"))
conf_url = OmegaConf.load(os.path.join(conf_dir, "config_url.yaml"))
conf_team = OmegaConf.load(os.path.join(conf_dir, "config_team.yaml"))
config = OmegaConf.merge(conf_cli, conf_exec, conf_path, conf_url, conf_team)

In [6]:
lake_dir = "../data/lake"
project_id = 'baseball-analytics-311804'

In [3]:
# 試合結果
df_lake_game = pd.read_csv(os.path.join(lake_dir, "lake_game.tsv"), sep="\t")

In [ ]:
sql = """
SELECT * FROM `bigquery-public-data.austin_311.311_service_requests` LIMIT 100
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

In [23]:
df_upload = df_lake_game.copy()
use_cols = ["game_id", "game_date", "score_top", "team_top_id", "exec_datetime"]
df_upload = df_upload[use_cols]

# exec_datetime
if "exec_datetime" in df_upload.columns:
    df_upload["exec_datetime"] = pd.to_datetime(df_upload["exec_datetime"])
else:
    pass

# game_date
if "game_date" in df_upload.columns:
    df_upload["game_date"] = pd.to_datetime(df_upload["game_date"]).dt.date
else:
    pass

db_name = "ba_tmp"
table_name = "lake_game"
table_id = db_name + "." + table_name
if_exists = "replace"
schema_dict = [
    {"name": "game_id", "type": "STRING"},
    {"name": "game_date", "type": "DATE"},
    {"name": "score_top", "type": "INTEGER"},
    {"name": "team_top_id", "type": "STRING"},
    {"name": "exec_datetime", "type": "DATETIME"},
]

# upload
pandas_gbq.to_gbq(
    df_upload, 
    destination_table=table_id, 
    project_id=project_id,
    if_exists=if_exists,
    table_schema=schema_dict,
    )

In [18]:
use_cols = ["game_id", "game_date", "score_top", "team_top_id", "exec_datetime"]
schema_dict  [
    {"name": "game_id", "type": "STRING"},
    {"name": "game_date", "type": "DATE"},
    {"name": "score_top", "type": "INTEGER"},
    {"name": "team_top_id", "type": "STRING"},
    {"name": "exec_datetime", "type": "DATETIME"},
]

,game_id,game_date,game_start_time,game_time,game_series,game_status,game_result,team_top_id,team_top_name,team_bottom_id,...,picher_lose_id,picher_save_id,starting_picher_bottom_id,starting_picher_top_id,audience_num,umpire_plate,umpire_first,umpire_second,umpire_third,exec_datetime
0,npb2021000096,2021-03-26,17:30,3時間36分,pennant_race,finish,top_win,npb5,阪神,npb2,...,npb1800033,npb1600013,npb1200062,npb1200076,"9,984人",深谷,本田,有隅,橋本,2021-04-06 02:33:17
1,npb2021000095,2021-03-26,17:45,3時間28分,pennant_race,finish,bottom_win,npb3,DeNA,npb1,...,npb1200049,NaN,npb1600081,npb1200041,"9,991人",牧田,山路,笠原,柳田,2021-04-06 02:33:17
2,npb2021000097,2021-03-26,18:00,2時間54分,pennant_race,finish,top_win,npb4,中日,npb6,...,npb1700070,npb1300066,npb1200055,npb1300057,"16,452人",土山,吉本,眞鍋,福家,2021-04-06 02:33:17
3,npb2021000098,2021-03-26,16:00,3時間40分,pennant_race,finish,bottom_win,npb8,日本ハム,npb376,...,npb1100099,NaN,npb1100099,npb500014,"14,559人",津川,村山,川口,敷田,2021-04-06 02:33:17
4,npb2021000099,2021-03-26,17:45,2時間30分,pennant_race,finish,bottom_win,npb11,オリックス,npb7,...,npb1600153,npb1200082,npb1600153,npb1400182,"9,599人",木内,秋村,原,西本,2021-04-06 02:33:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,npb2021000034,2021-03-09,13:00,2時間32分,pre_season_match,finish,bottom_win,npb9,ロッテ,npb376,...,npb1200094,npb2000124,npb1800052,npb700065,"1,114人",本田,長川,鈴木,山路,2021-04-06 09:28:29
71,npb2021000035,2021-03-09,13:00,2時間44分,pre_season_match,finish,top_win,npb7,西武,npb4,...,npb1800061,NaN,npb1800080,npb1800061,"1,439人",木内,市川,古賀,秋村,2021-04-06 09:28:29
72,npb2021000037,2021-03-09,13:00,3時間9分,pre_season_match,finish,bottom_win,npb6,広島,npb5,...,npb1600121,npb1300076,npb1600121,npb1900044,"5,131人",丹波,福家,土山,山本貴,2021-04-06 09:28:29
73,npb2021000036,2021-03-09,18:00,2時間46分,pre_season_match,finish,bottom_win,npb2,ヤクルト,npb11,...,npb2000025,npb1800101,npb1800143,npb1700109,"4,466人",飯塚,眞鍋,芦原,橘高,2021-04-06 09:28:29
